### **HugginFace - Transformers - Training a Model** ###

Anteriormente hemos utilizado un modelo preentrenado para hacer una clasificación de nuestros textos de entrada como Positivo o Negativo. En este caso, vamos a entrenar un modelo para poder clasificar nuestros textos por número de Estrellas. Para ellos, tomaremos un modelo y lo entrenaremos con un [Dataset](https://huggingface.co/datasets) escogido a tal efecto.

## *Modulos:* ##

In [ ]:
# Antes de ejecutar la instrucción de instalación de datasets, es neceario desinstalar pyarrow (la versión preinstalada en Colab no es complatible)
# Tras desistalar, es necesario Reiniciar la sesión en el entorno de ejecución
!pip uninstall -y pyarrow

In [10]:
# Tras la instalación, es necesario Reiniciar la sesión en el entorno de ejecución
!pip install transformers evaluate datasets accelerate -q

In [1]:
from datasets import load_dataset

## *Dataset:* ##

Vamos a trabajar con el Dataset "yelp_review_full", que contiene comentarios sobre películas y su clasificación en 1-5 estrellas.

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset

In [ ]:
dataset['train'][0] # Visualizamos el primer comentario del Conjunto de Datos, clasificado con 4 estrellas.

Dado que el Dataset con el que vamos a trabajar es muy grande, existe la posibilidad de comenzar a trabajar con un subconjunto de los datos para hacer pruebas y posteriormente entrenar el modelo con el conjunto de datos completo.

In [4]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(80))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(40))

## *Tokenizar el Dataset:* ##

Antes de pasarle los datos de entrada al modelo, es necesario Tokenizar los textos de entrada. Para ello, vamos a utilizar el modelo "bert-base-cased".

In [5]:
from transformers import AutoTokenizer

In [6]:
modelo = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(modelo)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

## *Modelo:* ##

Creamos el modelo "bert-base-cased".

In [7]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=5)

## *Entrenamiento:* ##

Vamos a entrenar el modelo haciendo uso de los datos disponibles en el Dataset.

In [9]:
import numpy as np
import evaluate

from transformers import TrainingArguments, Trainer

In [10]:
metric = evaluate.load("accuracy")

In [11]:
# Creamos la función compute_metrics que nos devolverá la exactitud de las predicciones
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Definimos los [argumentos](https://huggingface.co/transformers/v4.2.2/main_classes/trainer.html) del entrenamiento

In [12]:
training_args = TrainingArguments(
    'clasificador_estrellas',
    evaluation_strategy="steps",
    logging_steps=5,
    num_train_epochs=1,
    push_to_hub=True,
)

Instanciamos el objeto Trainer

In [13]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,
)

Finalmente, entrenamos el modelo.

In [ ]:
# Entrenamiento
trainer.train()

## *Guardar el modelo en local:* ##

In [ ]:
trainer.save_model("./modelo")

## *Subir el modelo a Hugging Face:* ##

In [ ]:
trainer.push_to_hub()
tokenizer.push_to_hub('Pegaso183/clasificador_estrellas')

PENDIENTE TRABAJAR EN LA MEJORA DEL MODELO

* Mejorar la eficiencia del entrenamiento mediante Lora.
* Incrementar los datos del Dataset de entrada utilizados.
* Incrementar el número de épocas de entrenamiento.
* Mejorar la parametrización del entrenamiento.